# Steps in a Data Collection Project

1. Identify the data source, requirements
*     Import libraries
*     Define constants, control variables and import credentials

2. Establish connection, collect sample data

3. Identify data format

4. Transform data for storage

5. Create function for bulk collection, transformation

6. Save data for later analysis

# 1. Identify the data source, requirements

FBI Webpage (tabular data): https://ucr.fbi.gov/crime-in-the-u.s/
OJJDP Webpage (CSV files available): https://www.ojjdp.gov/ojstatbb/crime/ucr.asp

Account/credentials required: n/a

    Import libraries

In [1]:
from bs4 import BeautifulSoup
import requests

import re

import pandas as pd

    Declare constants and control variables

In [2]:
MIN_YEAR = 2009
MAX_YEAR = 2019
FILE_PREFIX = "OJJDP_"

FBI_MIN_PUB_YEAR = 2017
FBI_MAX_PUB_YEAR = 2019

DATA_PATH = '../../data/'

# 2. Establish connection, collect sample data

In [3]:
OJJDP_permalink = "https://www.ojjdp.gov/ojstatbb/crime/ucr.asp"
OJJDP_query = "?selYrs="
year = 2019

url = OJJDP_permalink + OJJDP_query + str(year)
html = requests.get(url).text
OJJDP = BeautifulSoup(html, 'html5lib')

# 3. Identify data format

In [4]:
#syntax for class (class is a protected keyword in python)
#soup('element', class_=re.compile('classname')

table_id = "tbldata"

columns = [header.text for header in OJJDP.find('table', id=table_id).thead.findChildren('th')]

# remove newline characters \n
columns = [re.sub(r'\n', '', column) for column in columns]

# remove starting spaces
columns = [re.sub('^(\s*)', '', column) for column in columns]

# remove trailing spaces
columns = [re.sub('(\s*)$', '', column) for column in columns]
columns

['Offenses',
 'All ages',
 '0 to 17',
 '18 & older',
 '10 to 17',
 '0 to 14',
 '15 to 17',
 '18 to 20',
 '21 to 24',
 '25 & older']

In [5]:
df = pd.DataFrame(columns =columns, dtype = str)

In [6]:
# for each stats row in the table, pull out the stats
i = 0
lst = []

for row in OJJDP.find('table', id=table_id).tbody.findChildren('tr'):
    lst = []
    [lst.append(stats.text) for stats in row]
    i += 1
    
    df = df.append(pd.Series(lst, index = df.columns), ignore_index=True)
df

,Offenses,All ages,0 to 17,18 & older,10 to 17,0 to 14,15 to 17,18 to 20,21 to 24,25 & older
0,All offenses,"10,085,210","696,620","9,388,590","692,970","224,410","472,210","807,210","1,197,810","7,383,560"
1,Murder and nonnegligent manslaughter,"11,060",860,"10,200",860,110,760,"1,720","1,840","6,640"
2,Rape,NA,NA,NA,NA,NA,NA,NA,NA,NA
3,Robbery,"74,550","16,080","58,460","16,070","3,490","12,590","12,440","10,440","35,590"
4,Aggravated assault,"385,280","27,070","358,210","26,870","9,530","17,530","27,080","46,580","284,550"
5,Burglary,"171,590","20,700","150,890","20,570","6,860","13,840","15,890","19,700","115,300"
6,Larceny-theft,"813,070","83,690","729,380","83,380","24,860","58,830","76,410","87,050","565,930"
7,Motor vehicle theft,"80,640","13,610","67,030","13,600","3,670","9,940","7,620","9,310","50,100"
8,Arson,"9,070","1,800","7,270","1,730","1,020",780,530,720,"6,020"
9,Simple assault,"1,025,710","126,130","899,580","125,080","55,190","70,950","67,590","115,930","716,050"


# 4. Transform data for storage

In [7]:
def clean_columns(columns: list):
    # remove newline characters \n
    columns = [re.sub(r'\n', '', column) for column in columns]

    # remove starting spaces
    columns = [re.sub('^(\s*)', '', column) for column in columns]

    # remove trailing spaces
    return [re.sub('(\s*)$', '', column) for column in columns]

# 5. Create function for bulk collection, transformation

In [8]:
def oojdp_table_to_df(year: int):
    OJJDP_permalink = "https://www.ojjdp.gov/ojstatbb/crime/ucr.asp"
    OJJDP_query = "?selYrs="
      
    url = OJJDP_permalink + OJJDP_query + str(year)
    html = requests.get(url).text
    OJJDP = BeautifulSoup(html, 'html5lib')
    
    table_id = "tbldata"

    columns = [header.text for header in OJJDP.find('table', id=table_id).thead.findChildren('th')]
    columns = clean_columns(columns)
    
    df = pd.DataFrame(columns =columns, dtype = str)
    
    # for each stats row in the table, pull out the stats
    i = 0
    lst = []

    for row in OJJDP.find('table', id=table_id).tbody.findChildren('tr'):
        lst = []
        [lst.append(stats.text) for stats in row]
        i += 1

        df = df.append(pd.Series(lst, index = df.columns), ignore_index=True)

    return df

    Scrape FBI state-level data

In [12]:
def fbi_state_to_df(year: int):
    '''
    
    '''
    if (year < FBI_MIN_PUB_YEAR or year > FBI_MAX_PUB_YEAR):
        print("Crime data has not been published for this year, no dataframe returned")
        return
    
    template_url = "https://ucr.fbi.gov/crime-in-the-u.s/{year}/crime-in-the-u.s.-{year}/tables/table-11/table-11.xls/view"
    url = template_url.format(year = year)
    html = requests.get(url).text
    fed_soup = BeautifulSoup(html, 'html5lib')

    table_id = "table-data-container"
    
    if (year == 2019):
        #syntax for class (class is a protected keyword in python)
        #soup('element', class_=re.compile('classname'))
        states_links = {anchor.text: anchor['href'] for anchor in fed_soup.find('div', {"class": "secardarywrapper"}).findChildren('a')}
    else:
        states_links = {anchor.text: anchor['href'] for anchor in fed_soup.find('div', id='content_main').findChildren('a')}

    # not every table lists unit office, we will drop this column
    unit_office = 2
    
    # grab one state, url pair to create dataframe columns
    state_link = states_links.popitem()

    html = requests.get(state_link[1]).text
    state_soup = BeautifulSoup(html, 'html5lib')

    columns = [
        'State',
        'State/Tribal/Other',
        'Agency',
        'Unit/Office',
        'Violent crime',
        'Murder and nonnegligent manslaughter',
        'Rape',
        'Robbery',
        'Aggravated assault',
        'Property crime',
        'Burglary',
        'Larceny-theft',
        'Motor vehicle theft',
        'Arson']

    df = pd.DataFrame(columns = columns, dtype = str)
    
    # push the state pair back on the dict, for processing
    states_links[state_link[0]] = state_link[1]

    for key in states_links:
        
        url = states_links[key]
        html = requests.get(url).text
        state_soup = BeautifulSoup(html, 'html5lib')   
        
        state = key
        row_type = ""
        row_office = ""
        row_agency = ""

        for row in state_soup.find('div', id=table_id).tbody.findChildren('tr'):
            lst = []
            lst.append(state)

            # tables are varied on the site, our dataframe expects 13 columns
            # some tables keep values in table-header tags <th> while others use table-divs <td>
            # using this information we can fill in missing values from above rows, or impute
            columns = len(row.findChildren(['th', 'td']))
            children = row.findChildren(['th', 'td'])

            if (columns == 11):
                # we must impute type (from above rows)
                lst.append(row_type)
                # first value in table
                lst.append(re.sub(r'\n', '', children[0].text))
                # we must impute Unit/Office
                lst.append(row_office)
                # subsequent values in table
                [lst.append(re.sub(r'\n', '', stats.text)) for stats in children[1:]]

            elif (columns == 12):
                # we must store value of type, for subsequent rows
                row_type = re.sub(r'\n', '', children[0].text)
                # we must impute Unit/Office
                lst.append(row_type)
                # second value in table
                lst.append(re.sub(r'\n', '', children[1].text))
                # we must impute Unit/Office
                lst.append(row_office)
                # subsequent values in table
                [lst.append(re.sub(r'\n', '', stats.text)) for stats in children[2:]]

            else:
                # we must store value of type, for subsequent rows
                row_type = re.sub(r'\n', '', children[0].text)
                [lst.append(re.sub(r'\n', '', stats.text)) for stats in children]

            df = df.append(pd.Series(lst, index = df.columns), ignore_index=True)

    return df    

# 6. Save data for later analysis

    Capture and save OJJDP tables to csv files

In [ ]:
for year in range(MIN_YEAR, MAX_YEAR):
    filename = FILE_PREFIX + str(year) + ".csv"
    df = oojdp_table_to_df(year)
    
    df.to_csv(DATA_PATH + filename, index=False)

    Save 2019 FBI table to .csv

In [13]:
fbi_state_2019_df = fbi_state_to_df(2019)

fbi_state_2019_df.to_csv(DATA_PATH + "fbi_state_2019.csv", index=False)

    save FBI tables to csv files

In [15]:
# for year in range(FBI_MIN_PUB_YEAR, FBI_MAX_PUB_YEAR+1):
#     filename = "FBI_State_" + str(year) + ".csv"
#     print("Processing data from {year}".format(year=year))
    
#     df = fbi_state_to_df(year)

#     print("Saving file {filename}".format(filename=filename))
#     df.to_csv(DATA_PATH + filename, index=False)